# Workshop: From Zero to Hero
Cómo hacer un chatbot con GPT y Langchain, y montar una UI de prototipo

AI Hackathon by CommunityOS - OpenAI - Fintual

## Quién es Rodrigo Basoalto
- Ingeniero de Software, MS en Ciencia de la Computación
- ~13 años full-time en la industria, muchos más como _hobbyist_
- Lidero el equipo de Fintual AI
- Antes: Google, Evernote, Groupon, Synopsys

# Fintual 💙 AI
Qué es Fintual? revisa [fintual.com](https://fintual.com)

Qué hacemos con LLMs?
- Trabajando en un asesor de inversiones por WhatsApp, pruébenlo! https://fintu.al/hola (por ahora sólo +1🇺🇸, +52🇲🇽, +56🇨🇱)
- "Copilot" para el chat de soporte, esperamos pronto empezar a responder directamente lo que podamos
    - Evaluación, feedback, prompt engineering, ...
- Muuuuchos procesos internos

Además de _otras_ formas de AI/ML, ej: CTGAN para asset allocation https://doi.org/10.1080/14697688.2024.2329194

# Público Objetivo
Para quién es este workshop?
- Tienes ideas pero no sabes cómo implementarlas en la práctica
- Has usado LLMs para hacer _cosas_
- Quizá has usado las APIs de OpenAI u otros
- Sabes al menos un poco de Python
    - Casi todo lo del workshop es aplicable 1:1 en JS también
    - Conceptualmente es aplicable en cualquier lenguaje (pero Langchain está solo en Py/JS)

## Estructura del Workshop
Vamos a ir paso a paso mostrando, en concreto, las partes para armar un chatbot. Usaremos Python + OpenAI + Langchain + Gradio.
- API de OpenAI para llamadas simples
- Introducción a Langchain
    - Un chatbot básico
    - Agentes
    - Tools
- Armando un prototipo completo con Gradio

## Llamando a GPT directamente
Necesitamos tener una cuenta en OpenAI (ojo: platform, no ChatGPT), y tener `OPENAI_API_KEY` definida como variable de entorno.

In [1]:
import os
os.environ['OPENAI_API_KEY'][:3]

'sk-'

In [2]:
from openai import OpenAI

client = OpenAI()

system_prompt = "You're a seasoned chef working as a helpful mentor to cooking enthusiasts."

result = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": "Tengo huevos, mantequilla, cilantro, y cebollín, qué comida rica puedo hacer?",
        },
    ],
    model="gpt-4o",
)
result

ChatCompletion(id='chatcmpl-9rYJHiRw7wlYIzCTR5g02WFg6ti3U', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='¡Con esos ingredientes puedes preparar una deliciosa omelette! Aquí tienes una receta sencilla:\n\n### Omelette con cilantro y cebollín\n\n#### Ingredientes:\n- 3 huevos\n- 1 cucharada de mantequilla\n- 2 cucharadas de cilantro fresco, picado\n- 2 cucharadas de cebollín, picado\n- Sal y pimienta al gusto\n\n#### Instrucciones:\n\n1. **Preparar los ingredientes**: Pica finamente el cilantro y el cebollín. Reserva una pequeña cantidad de ambos para decorar al final, si lo deseas.\n  \n2. **Batir los huevos**: En un bol, rompe los huevos y bátelos con un tenedor o un batidor de mano hasta que queden bien mezclados. Agrega una pizca de sal y pimienta al gusto.\n\n3. **Calentar la sartén**: Coloca una sartén antiadherente a fuego medio y derrite la mantequilla, asegurándote de que cubra toda la superficie de la sartén.\n\n4. **Cocin

In [3]:
print(result.choices[0].message.content)

¡Con esos ingredientes puedes preparar una deliciosa omelette! Aquí tienes una receta sencilla:

### Omelette con cilantro y cebollín

#### Ingredientes:
- 3 huevos
- 1 cucharada de mantequilla
- 2 cucharadas de cilantro fresco, picado
- 2 cucharadas de cebollín, picado
- Sal y pimienta al gusto

#### Instrucciones:

1. **Preparar los ingredientes**: Pica finamente el cilantro y el cebollín. Reserva una pequeña cantidad de ambos para decorar al final, si lo deseas.
  
2. **Batir los huevos**: En un bol, rompe los huevos y bátelos con un tenedor o un batidor de mano hasta que queden bien mezclados. Agrega una pizca de sal y pimienta al gusto.

3. **Calentar la sartén**: Coloca una sartén antiadherente a fuego medio y derrite la mantequilla, asegurándote de que cubra toda la superficie de la sartén.

4. **Cocinar la mezcla de huevos**: Vierte los huevos batidos en la sartén caliente. Deja que se cocinen unos momentos sin mover. Luego, usando una espátula, mueve suavemente los bordes haci

### Un desvío temporal: observabilidad con Langsmith
Langchain ofrece un servicio para observabilidad y evaluación de aplicaciones con LLMs (y otras), podemos probar la versión más simple de una traza en Langsmith

Necesitamos definir las variables de ambiente `LANGCHAIN_TRACING_V2=true` y `LANGCHAIN_API_KEY=lsv2_...`

In [4]:
from langsmith.wrappers import wrap_openai
from langsmith import traceable

# Wrap OpenAI client to trace its inputs/outputs
ls_client = wrap_openai(client)

# We can decorate any function with @traceable too!
@traceable
def ask_chef(message: str) -> str | None:
    result = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": message,
            },
        ],
        model="gpt-4o",
    )
    return result.choices[0].message.content

In [5]:
print(ask_chef("I have a whole chicken, and I want to cook it as quickly as possible. Any suggestions?"))

Absolutely! If you're looking to cook a whole chicken quickly, here are a few methods that can help:

### Spatchcock (Butterflied) Chicken:
- **Time Required**: Approximately 45 minutes.
- **Method**:
1. **Preheat Oven**: Set your oven to 450°F (230°C).
2. **Prepare Chicken**: Place the chicken breast-side down. Use kitchen shears to cut along both sides of the backbone and remove it. Flip the chicken over and press down on the breastbone to flatten it out completely.
3. **Season**: Rub the chicken with olive oil, and season generously with salt, pepper, and any herbs or spices you like.
4. **Cook**: Place the chicken on a baking sheet or in a large cast-iron skillet, breast side up. Roast in the preheated oven for about 35-45 minutes, until the internal temperature reaches 165°F (75°C).
5. **Rest & Serve**: Let the chicken rest for 10 minutes before carving.

### Chicken in a Pressure Cooker (Instant Pot):
- **Time Required**: Approximately 30-35 minutes.
- **Method**:
1. **Season Chi

Ahora podemos ir a [Langsmith](https://smith.langchain.com) y revisar nuestras trazas

## Langchain
Empezó como un framework para construir prompts y darle _superpoderes_ a los LLMs. En la época en que no había chat models, ni function calling. Con puro prompting te daba una interfaz que permitía crear agentes con herramientas, chatbots con historial, etc.

Hoy ya no es tan necesario todo eso pero sigue siendo una forma relativamente limpia y ordenada de combinar bloques simples para armar estructuras más complejas.

### Langchain básico
Vamos a construir lo mismo de antes pero con las primitivas de Langchain

In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

Ahora lo invocamos con los mensajes de sistema y usuario...

In [7]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(system_prompt),
    HumanMessage("cómo preparo un barros luco?"),
]

result = model.invoke(messages)
result

AIMessage(content='¡Claro! El Barros Luco es un sándwich chileno clásico, simple y delicioso. Aquí tienes una receta básica para prepararlo:\n\n### Ingredientes:\n- 1 pan de marraqueta (puedes usar otro tipo de pan si prefieres, como pan francés o pan ciabatta)\n- 150 gramos de carne de res (puedes usar bistec delgado o lomo)\n- 2 a 3 rebanadas de queso (generalmente se usa queso mantecoso, pero puedes usar otro queso derretible como queso cheddar o mozzarella)\n- Sal y pimienta al gusto\n- Aceite o mantequilla para cocinar\n\n### Instrucciones:\n\n1. **Preparar la Carne:**\n   - Si la carne no está en filetes delgados, puedes golpearla un poco para hacerla más delgada y tierna.\n   - Sazona la carne con sal y pimienta al gusto.\n\n2. **Cocinar la Carne:**\n   - En una sartén caliente, añade un poco de aceite o mantequilla.\n   - Cocina la carne a fuego medio-alto hasta que esté dorada y cocida a tu gusto. Esto normalmente toma unos 2-3 minutos por lado, dependiendo del grosor.\n\n3. *

In [8]:
print(result.content)

¡Claro! El Barros Luco es un sándwich chileno clásico, simple y delicioso. Aquí tienes una receta básica para prepararlo:

### Ingredientes:
- 1 pan de marraqueta (puedes usar otro tipo de pan si prefieres, como pan francés o pan ciabatta)
- 150 gramos de carne de res (puedes usar bistec delgado o lomo)
- 2 a 3 rebanadas de queso (generalmente se usa queso mantecoso, pero puedes usar otro queso derretible como queso cheddar o mozzarella)
- Sal y pimienta al gusto
- Aceite o mantequilla para cocinar

### Instrucciones:

1. **Preparar la Carne:**
   - Si la carne no está en filetes delgados, puedes golpearla un poco para hacerla más delgada y tierna.
   - Sazona la carne con sal y pimienta al gusto.

2. **Cocinar la Carne:**
   - En una sartén caliente, añade un poco de aceite o mantequilla.
   - Cocina la carne a fuego medio-alto hasta que esté dorada y cocida a tu gusto. Esto normalmente toma unos 2-3 minutos por lado, dependiendo del grosor.

3. **Derretir el Queso:**
   - Coloca las 

(Y si vamos a mirar a [Langsmith](https://smith.langchain.com)?)

OK pero hasta ahora no ha simplificado mucho... podemos empezar a combinar bloques, como por ejemplo para extraer el mensaje de la respuesta

In [9]:
from langchain_core.output_parsers import StrOutputParser

chain = model | StrOutputParser()

print(chain.invoke(messages))

¡Claro! El Barros Luco es un sándwich chileno muy popular, conocido por su sencillez y delicioso sabor. Aquí te dejo una receta básica para prepararlo:

### Ingredientes:
- 2 panes (puede ser marraqueta, hallulla o pan de tu preferencia)
- 200 gramos de carne de vacuno (puede ser lomo, posta o cualquier corte tierno)
- 2-4 lonchas de queso (puedes usar queso mantecoso, gouda o el que prefieras que se derrita bien)
- Sal y pimienta al gusto
- Aceite o mantequilla para cocinar

### Instrucciones:
1. **Preparar la carne**: Corta la carne en tiras finas o en filetes delgados. Si prefieres, puedes pedirle al carnicero que lo haga por ti.

2. **Cocinar la carne**: En una sartén grande, calienta un poco de aceite o mantequilla a fuego medio-alto. Agrega la carne y cocina hasta que esté dorada por ambos lados, unos 3-5 minutos dependiendo del grosor de la carne. Sazona con sal y pimienta al gusto.

3. **Preparar el queso**: Mientras la carne se cocina, puedes calentar las lonchas de queso en u

Y aprovechemos de pasar por Prompt Templates, que nos permiten armar prompts más dinámicos

In [10]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    # Note that Langchain can almost always take (type, content) tuples in place of the specific message object types
    ("system", system_prompt + " You always answer in {language}."),
    ("human", "{input}"),
])

In [11]:
chain = prompt_template | model | StrOutputParser()

print(chain.invoke({"language": "japanese", "input": "how do I make sushi?"}))

寿司の作り方についてお教えしますね。

### 材料：
- 寿司米（コシヒカリなど）
- 寿司酢（米酢、砂糖、塩を混ぜたもの）
- 海苔（巻き寿司用）
- 新鮮な魚（サーモン、マグロなど）
- 野菜（キュウリ、アボカドなど）
- 醤油
- わさび
- ガリ（甘酢生姜）

### 手順：

1. **寿司米の準備**
   1. 寿司米をよく洗い、透明になるまで水で研ぎます。
   2. 炊飯器で適量の水と共に炊きます。
   3. 炊き上がったご飯をボウルに移し、寿司酢を少しずつ加えながら、しゃもじで切るように混ぜます。ご飯が冷めるまで扇で風を当てながら混ぜ続けます。

2. **具材の準備**
   1. 新鮮な魚を薄くスライスします。
   2. キュウリやアボカドなどの野菜を細長く切ります。

3. **巻き寿司の作り方**
   1. 巻きすの上に海苔を置きます。
   2. 海苔の上に寿司米を広げます。端の方は少し残すようにします。
   3. 中央に魚や野菜を並べます。
   4. 巻きすを使って、海苔を巻き込みながらしっかりと巻きます。
   5. 巻き終わったら、適当な大きさに切ります。

4. **握り寿司の作り方**
   1. 手に水をつけ、寿司米を適量手に取ります。
   2. 軽く握って小さな俵型に整えます。
   3. スライスした魚を上に乗せ、軽く押さえます。

### 提供方法：
- お皿に寿司を並べ、醤油、わさび、ガリを添えて提供します。

おいしい寿司を作るためには、新鮮な食材と丁寧な準備が重要です。ぜひ挑戦してみてください！


## Hagamos un Chatbot
Qué gracia tiene un chatbot que no tengamos aun? Memoria!

In [12]:
from langchain_core.messages import AIMessage

messages = [
    SystemMessage("You're a simple companion. You go straight to the point and give very concise answers."),
    HumanMessage("Hola, me llamo Rodrigo"),
    AIMessage("Hola Rodrigo!"),
    HumanMessage("Cómo me llamo?"),
]
chain = model | StrOutputParser()
print(chain.invoke(messages))

Rodrigo.


Ahora... cómo hacemos que sea más interactivo?

In [13]:
from langchain_core.prompts import MessagesPlaceholder, HumanMessagePromptTemplate

history = [
    HumanMessage("Hola, me llamo Rodrigo"),
    AIMessage("Hola Rodrigo!"),
]

messages = [
    SystemMessage("You're a simple companion. You go straight to the point and give very concise answers."),
    # We need a place for the chat history until "now"
    MessagesPlaceholder(variable_name="history"),
    # And then the new user message
    HumanMessagePromptTemplate.from_template("{user_message}"),
]

chain = ChatPromptTemplate.from_messages(messages) | model | StrOutputParser()

@traceable
def chat(message: str) -> str:
    return chain.invoke({"history": history, "user_message": message})    

In [14]:
chat("cual es mi nombre?")

'Rodrigo.'

Pero nosotros tenemos que manejar la memoria "a mano", agregar los mensajes en la historia, etc, etc.

Langchain facilita todo eso, envolviendo un `Runnable` (e.g. una cadena) en algo que inyecta historia, y captura el _output_ para agregarlo a la historia para después: `RunnableWithMessageHistory`.

In [15]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory


chat_history = InMemoryChatMessageHistory()

def get_history() -> InMemoryChatMessageHistory:
    return chat_history

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(system_prompt),
    MessagesPlaceholder(variable_name="messages"),
])

base_chain = prompt | model
chain = RunnableWithMessageHistory(base_chain, get_history, input_messages_key="messages") | StrOutputParser()

In [16]:
print(chain.invoke({
    "messages":[
        HumanMessage(content="hola, quiero cocinar algo con huevos y papas y cebolla")
    ]
}))

¡Hola! Parece que tienes los ingredientes perfectos para preparar una clásica tortilla española o tortilla de patatas. Aquí tienes una receta sencilla para que puedas hacerla:

**Ingredientes:**
- 4-5 papas medianas
- 1 cebolla grande
- 6-7 huevos
- Aceite de oliva
- Sal al gusto

**Instrucciones:**

1. **Preparar las papas y la cebolla:**
   - Pela las papas y córtalas en rodajas finas.
   - Pela la cebolla y córtala en juliana (tiras finas).

2. **Freír las papas y la cebolla:**
   - En una sartén grande, calienta abundante aceite de oliva a fuego medio.
   - Añade las papas y la cebolla a la sartén y cocina hasta que estén blandas y ligeramente doradas. Remueve de vez en cuando para que no se peguen.
   - Una vez cocidas, retira las papas y la cebolla con una espumadera y escúrrelas bien para eliminar el exceso de aceite. Puedes ponerlas sobre papel absorbente.

3. **Preparar la mezcla de huevos:**
   - En un bol grande, bate los huevos con una pizca de sal.
   - Añade las papas y l

In [17]:
print(chain.invoke({
    "messages": [
        HumanMessage("y si tengo chorizo?"),
        HumanMessage("Ah no, me equivoqué, no tengo chorizo, pero tengo jamón"),
    ]
}))

¡No hay problema! El jamón también es una excelente adición a la tortilla española y le dará un sabor muy rico. Aquí te dejo una versión ajustada de la receta para incluir el jamón:

**Ingredientes:**
- 4-5 papas medianas
- 1 cebolla grande
- 6-7 huevos
- 100-150 gramos de jamón (puede ser jamón serrano o jamón cocido, según prefieras)
- Aceite de oliva
- Sal al gusto

**Instrucciones:**

1. **Preparar las papas y la cebolla:**
   - Pela las papas y córtalas en rodajas finas.
   - Pela la cebolla y córtala en juliana (tiras finas).

2. **Freír las papas y la cebolla:**
   - En una sartén grande, calienta abundante aceite de oliva a fuego medio.
   - Añade las papas y la cebolla a la sartén y cocina hasta que estén blandas y ligeramente doradas. Remueve de vez en cuando para que no se peguen.
   - Una vez cocidas, retira las papas y la cebolla con una espumadera y escúrrelas bien para eliminar el exceso de aceite. Puedes ponerlas sobre papel absorbente.

3. **Preparar el jamón:**
   - C

Excelente, ahora tenemos memoria y funciona como un chat interactivo.

Langchain tiene varias implementaciones de `ChatMessageHistory`, para guardar historial en bases de datos, en archivos, etc. Explorar esas queda como ejercicio para el lector.

Ahora veamos un ejemplo de cómo mantener varias sesiones en paralelo... podemos pasar configuración como input de la cadena, que podrá ser usada por sus componentes. Ejemplo:

In [18]:
from collections import defaultdict

histories: dict[str, InMemoryChatMessageHistory] = defaultdict(InMemoryChatMessageHistory)

def get_history_by_session_id(session_id: str) -> InMemoryChatMessageHistory:
    return histories[session_id]

chain = RunnableWithMessageHistory(base_chain, get_history_by_session_id, input_messages_key="messages") | StrOutputParser()

@traceable
def chat(session_id: str, message: str) -> str:
    config = {
        "configurable": {
            "session_id": session_id,
        }
    }
    return chain.invoke(
        {
            "messages": [HumanMessage(message)],
        },
        config=config,
    )

In [19]:
print(chat("rodrigo", "quiero cocinar pastel de choclo, qué ingredientes necesito comprar?"))

¡Claro! El pastel de choclo es un plato delicioso típico de la cocina chilena. Aquí tienes una lista de ingredientes que necesitarás para preparar un pastel de choclo tradicional:

### Ingredientes para la "pasta de choclo":
- 8-10 choclos (maíz) frescos
- 1 taza de leche
- 2 cucharadas de mantequilla
- 1 cucharadita de sal
- 1 cucharadita de azúcar (opcional)

### Ingredientes para el relleno:
- 500 gramos de carne molida (puede ser de res o una mezcla de res y cerdo)
- 1 cebolla grande, picada en cuadros pequeños
- 2 dientes de ajo, picados finamente
- 1 cucharadita de comino
- 1 cucharadita de ají de color (pimentón dulce)
- Sal y pimienta al gusto
- 2 cucharadas de aceite vegetal
- 2 huevos duros, cortados en cuartos
- 1 pechuga de pollo cocida y desmenuzada (opcional)
- 1/2 taza de aceitunas negras (sin carozo)
- 1/2 taza de pasas (opcional)

### Ingredientes adicionales:
- Azúcar para espolvorear sobre el pastel antes de hornear (opcional, pero tradicional en algunas recetas)

##

In [20]:
print(chat("pedro", "tengo platanos, naranjas, y kiwis, como hago tutti-fruti?"))

¡Claro! Hacer un tutti-fruti con plátanos, naranjas y kiwis es sencillo y delicioso. Aquí tienes una receta básica:

### Ingredientes:
- 2 plátanos
- 2 naranjas
- 3 kiwis
- Jugo de una naranja (opcional)
- Miel o azúcar al gusto (opcional)
- Hojas de menta para decorar (opcional)

### Instrucciones:

1. **Preparar las frutas:**
   - Pela los plátanos y córtalos en rodajas.
   - Pela las naranjas, retira las semillas y corta los gajos en trozos más pequeños.
   - Pela los kiwis y córtalos en rodajas o en trozos pequeños.

2. **Mezclar las frutas:**
   - Coloca todas las frutas cortadas en un tazón grande.
   - Si deseas, puedes añadir el jugo de una naranja para darle un toque extra de sabor y evitar que los plátanos se oxiden rápidamente.

3. **Endulzar (opcional):**
   - Si prefieres un tutti-fruti más dulce, puedes añadir una cucharada de miel o un poco de azúcar. Mezcla bien para que el endulzante se distribuya uniformemente.

4. **Refrigerar (opcional):**
   - Para disfrutarlo bien

In [21]:
print(chat("rodrigo", "no quedaba carne, lo puedo hacer veggie?"))

¡Claro que sí! Puedes hacer una versión vegetariana del pastel de choclo que sea igual de deliciosa. Aquí te dejo una receta adaptada sin carne:

### Ingredientes para la "pasta de choclo":
- 8-10 choclos (maíz) frescos
- 1 taza de leche
- 2 cucharadas de mantequilla (puedes usar margarina o aceite de coco para una versión vegana)
- 1 cucharadita de sal
- 1 cucharadita de azúcar (opcional)

### Ingredientes para el relleno vegetariano:
- 1 cebolla grande, picada en cuadros pequeños
- 2 dientes de ajo, picados finamente
- 1 zanahoria grande, rallada
- 1 taza de champiñones, picados
- 1 pimiento rojo, picado en cubos pequeños
- 1 taza de espinacas frescas, picadas
- 1 taza de choclo (maíz) desgranado
- 1 cucharadita de comino
- 1 cucharadita de ají de color (pimentón dulce)
- Sal y pimienta al gusto
- 2 cucharadas de aceite vegetal
- 2 huevos duros, cortados en cuartos (opcional, omitir para una versión vegana)
- 1/2 taza de aceitunas negras (sin carozo)
- 1/2 taza de pasas (opcional)

#

## Agentes
Ya sabemos hacer chatbots! Ahora vamos un paso más allá: si queremos que el chatbot pueda **hacer** cosas, necesitamos reestructurar el flujo. Podemos usar _function calling_ para que el LLM pueda tomar acciones en respuesta al usuario, y luego re-evaluar en base al resultado de esas acciones.

### Tools
Empecemos por explorar las herramientas, y qué mejor para partir que usar alguna herramienta predefinida

In [22]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

wikipedia_api_wrapper = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=5000)
wikipedia_query = WikipediaQueryRun(api_wrapper=wikipedia_api_wrapper)

print(wikipedia_query.invoke({"query":"openai"}))

Page: OpenAI
Summary: OpenAI is an American artificial intelligence (AI) research organization founded in December 2015 and headquartered in San Francisco, California. Its mission is to develop "safe and beneficial" artificial general intelligence, which it defines as "highly autonomous systems that outperform humans at most economically valuable work". As a leading organization in the ongoing AI boom, OpenAI is known for the GPT family of large language models, the DALL-E series of text-to-image models, and a text-to-video model named Sora. Its release of ChatGPT in November 2022 has been credited with catalyzing widespread interest in generative AI.
The organization consists of the non-profit OpenAI, Inc., registered in Delaware, and its for-profit subsidiary OpenAI Global, LLC. Microsoft owns roughly 49% of OpenAI's equity, having invested US$13 billion. It also provides computing resources to OpenAI through its Microsoft Azure cloud platform.
In 2023 and 2024, OpenAI faced multiple

Esencialmente es una función con algunos metadatos para que el LLM la _entienda_

In [23]:
wikipedia_query.name, wikipedia_query.description, wikipedia_query.args_schema.schema()

('wikipedia',
 'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.',
 {'title': 'WikipediaQueryInput',
  'description': 'Input for the WikipediaQuery tool.',
  'type': 'object',
  'properties': {'query': {'title': 'Query',
    'description': 'query to look up on wikipedia',
    'type': 'string'}},
  'required': ['query']})

Podemos hacer nuestras propias herramientas con decoradores

In [24]:
from langchain_core.tools import tool

@tool
def weather(city_name: str) -> str:
    """Gets the weather forecast for a city."""
    match city_name.lower():
        case 'santiago':
            return 'cold'
        case 'valparaiso':
            return 'windy'
        case 'concepcion':
            return 'rainy'
        case _:
            return 'fair'

weather.name, weather.description, weather.args_schema.schema()

('weather',
 'Gets the weather forecast for a city.',
 {'title': 'weatherSchema',
  'description': 'Gets the weather forecast for a city.',
  'type': 'object',
  'properties': {'city_name': {'title': 'City Name', 'type': 'string'}},
  'required': ['city_name']})

O podemos hacer tools con clases derivadas de `BaseTool` "a mano"

In [25]:
from typing import Type
from langchain_core.tools import BaseTool
from langchain_core.pydantic_v1 import BaseModel, Field

class CompoundInterestCalculatorArgs(BaseModel):
    n_periods: int = Field(description="Number of periods to calculate for")
    rate: float = Field(description="Interest rate per period, in percentage points")
    initial_amount: float = Field(description="Amount at the beginning of the first period")

class CompoundInterestCalculator(BaseTool):
    args_schema: Type[BaseModel] = CompoundInterestCalculatorArgs
    name = 'compound_interest_calculator'
    description = 'Calculates the compound interest of an initial amount over a number of periods at a specified rate'

    def _run(self, n_periods: int, rate: float, initial_amount: float) -> str:
        return str(initial_amount*(1+rate/100)**n_periods)

compound_interest_calc = CompoundInterestCalculator()

compound_interest_calc.name, compound_interest_calc.description, compound_interest_calc.args_schema.schema()

('compound_interest_calculator',
 'Calculates the compound interest of an initial amount over a number of periods at a specified rate',
 {'title': 'CompoundInterestCalculatorArgs',
  'type': 'object',
  'properties': {'n_periods': {'title': 'N Periods',
    'description': 'Number of periods to calculate for',
    'type': 'integer'},
   'rate': {'title': 'Rate',
    'description': 'Interest rate per period, in percentage points',
    'type': 'number'},
   'initial_amount': {'title': 'Initial Amount',
    'description': 'Amount at the beginning of the first period',
    'type': 'number'}},
  'required': ['n_periods', 'rate', 'initial_amount']})

Bien, sabemos hacer herramientas, pero queremos que un LLM las llame... Para eso basta con presentárselas, y el modelo puede _decidir_ usarlas, i.e. producir un output que pide ejecutar una función/tool

In [26]:
tools = [wikipedia_query, weather, compound_interest_calc]
model_with_tools = model.bind_tools(tools)

messages = [
    SystemMessage("You're a helpful assistant"),
    HumanMessage("What's the weather in Santiago?"),
]

model_output = model_with_tools.invoke(messages)

model_output

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3gDJQ5NXlQuCv7wdGVXdTHRH', 'function': {'arguments': '{"city_name":"Santiago"}', 'name': 'weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 185, 'total_tokens': 200}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_4e2b2da518', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7f966176-2571-402b-b7eb-068639dbcb17-0', tool_calls=[{'name': 'weather', 'args': {'city_name': 'Santiago'}, 'id': 'call_3gDJQ5NXlQuCv7wdGVXdTHRH', 'type': 'tool_call'}], usage_metadata={'input_tokens': 185, 'output_tokens': 15, 'total_tokens': 200})

In [27]:
model_output.tool_calls

[{'name': 'weather',
  'args': {'city_name': 'Santiago'},
  'id': 'call_3gDJQ5NXlQuCv7wdGVXdTHRH',
  'type': 'tool_call'}]

Aquí la respuesta del modelo dice "necesito llamar a `weather` para responder"... llamémoslo y le damos el resultado al model de nuevo

In [28]:
tool_message = weather.invoke(model_output.tool_calls[0])
tool_message

ToolMessage(content='cold', name='weather', tool_call_id='call_3gDJQ5NXlQuCv7wdGVXdTHRH')

In [29]:
messages.extend([model_output, tool_message])

final_answer = model_with_tools.invoke(messages)
final_answer.content

'The current weather in Santiago is cold.'

Perfecto, ya sabemos manejar llamadas a LLMs con herramientas. Pero si queremos encapsularlo todo en algo más manejable? Aquí es cuando necesitamos un ✨Agente✨

### Agente
Un agente no es más que una encapsulación de un LLM, tools, y una forma de ejecutar todo alimentándole el resultado de vuelta al LLM.

Hay muchas estructuras de agentes, una muy simple y de los más comunes es ReAct (Reasoning and Action).

Para implementar esos flujos con Langchain, se pueden hacer con combinaciones de cadenas y código (_a la antigua_), o con LangGraph, donde uno construye el flujo en forma de grafo que luego se puede ejecutar como parte de una cadena normal. LangGraph ya tiene una implementación de agente ReAct, estructuras de agentes más complejas quedan como ejercicio para el lector.

In [30]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(model, tools)

result = agent.invoke({"messages": [
    HumanMessage("How much money would I have if I deposit 100k for 5 years with a 5% APR, using compound interest"),
]})

In [31]:
result

{'messages': [HumanMessage(content='How much money would I have if I deposit 100k for 5 years with a 5% APR, using compound interest', id='71220118-8b3b-449e-820b-e813cba60db3'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Tj20lyakQuUPaKiSrvyyW2xM', 'function': {'arguments': '{"n_periods":5,"rate":5,"initial_amount":100000}', 'name': 'compound_interest_calculator'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 200, 'total_tokens': 229}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_4e2b2da518', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8785f754-18fa-42bc-8dd4-c781120a9d5e-0', tool_calls=[{'name': 'compound_interest_calculator', 'args': {'n_periods': 5, 'rate': 5, 'initial_amount': 100000}, 'id': 'call_Tj20lyakQuUPaKiSrvyyW2xM', 'type': 'tool_call'}], usage_metadata={'input_tokens': 200, 'output_tokens': 29, 'total_tokens': 229}),
  ToolMessage(content='127628.15625000003', n

In [32]:
def print_messages(result):
    for message in result['messages']:
        print("=" * 10, message.type, "=" * 10)
        print(message.content or message.tool_calls)

In [33]:
print_messages(result)

========== human ==========
How much money would I have if I deposit 100k for 5 years with a 5% APR, using compound interest
========== ai ==========
[{'name': 'compound_interest_calculator', 'args': {'n_periods': 5, 'rate': 5, 'initial_amount': 100000}, 'id': 'call_Tj20lyakQuUPaKiSrvyyW2xM', 'type': 'tool_call'}]
========== tool ==========
127628.15625000003
========== ai ==========
If you deposit $100,000 for 5 years with a 5% annual percentage rate (APR) using compound interest, you would have approximately $127,628.16 at the end of the period.


Ahora nos falta la memoria! Lamentablemente con LangGraph no nos sirve exactamente lo que vimos antes, pero también soporta _checkpointing_ (guardar el estado del grafo, las entradas, los mensajes intermedios, los resultados, i.e. memoria) y también trae algunas implementaciones predefinidas.

In [34]:
from langgraph.checkpoint import MemorySaver

memory = MemorySaver() # saves checkpoints in-memory... SQLite implementation available too!
agent = create_react_agent(model, tools, checkpointer=memory)

# the checkpointer takes a RunnableConfig for "thread_id" to support multiple threads
config = {"configurable":{"thread_id": "rodrigo"}}

result = agent.invoke({"messages": [
    HumanMessage("quiénes son los candidatos para la elección presidencial de USA 2024?"),
]}, config=config)

In [35]:
print_messages(result)

========== human ==========
quiénes son los candidatos para la elección presidencial de USA 2024?
========== ai ==========
[{'name': 'wikipedia', 'args': {'query': '2024 United States presidential election candidates'}, 'id': 'call_AqoY7Iz423V3ImwxENZsobGJ', 'type': 'tool_call'}]
========== tool ==========
Page: 2024 United States presidential election
Summary: The 2024 United States presidential election will be the 60th quadrennial presidential election, set to be held on Tuesday, November 5, 2024. Voters in each state and the District of Columbia will choose a slate of electors to the U.S. Electoral College, who will then elect a president and vice president for a term of four years.
Incumbent president Joe Biden, a member of the Democratic Party, initially ran for re-election and became the party's presumptive nominee on March 12. However, following a poor performance in the June 2024 presidential debate and increasing age and health concerns, he withdrew on July 21 and endorsed Vi

In [36]:
result = agent.invoke({"messages": [HumanMessage('y como está el tiempo en Washington DC?')]}, config)

print_messages(result)

========== human ==========
quiénes son los candidatos para la elección presidencial de USA 2024?
========== ai ==========
[{'name': 'wikipedia', 'args': {'query': '2024 United States presidential election candidates'}, 'id': 'call_AqoY7Iz423V3ImwxENZsobGJ', 'type': 'tool_call'}]
========== tool ==========
Page: 2024 United States presidential election
Summary: The 2024 United States presidential election will be the 60th quadrennial presidential election, set to be held on Tuesday, November 5, 2024. Voters in each state and the District of Columbia will choose a slate of electors to the U.S. Electoral College, who will then elect a president and vice president for a term of four years.
Incumbent president Joe Biden, a member of the Democratic Party, initially ran for re-election and became the party's presumptive nominee on March 12. However, following a poor performance in the June 2024 presidential debate and increasing age and health concerns, he withdrew on July 21 and endorsed Vi

Sólo nos falta ver cómo darle un _system prompt_ al agente. En jerga de LangGraph, tenemos que manipular el estado inicial del grafo para que parta con un `SystemMessage`, pero basta con pasárselo a `create_react_agent` y ya.

In [37]:
memory = MemorySaver() # let's start fresh with an empty memory...

system_prompt = "You're a helpful agent. You always answer in Latin, regardless of the language used by the user."

agent = create_react_agent(model, tools, checkpointer=memory, state_modifier=system_prompt)

# the checkpointer takes a RunnableConfig param of "thread_id" to support multiple threads
config = {"configurable":{"thread_id": "rodrigo"}}

result = agent.invoke({"messages": [
    HumanMessage("cuánta plata tengo después de 10 años si deposito 1m con 5% de interés anual?"),
]}, config=config)

print_messages(result)

========== human ==========
cuánta plata tengo después de 10 años si deposito 1m con 5% de interés anual?
========== ai ==========
[{'name': 'compound_interest_calculator', 'args': {'n_periods': 10, 'rate': 5, 'initial_amount': 1000000}, 'id': 'call_PNybPPtZLEBh9obp0SVKZKWJ', 'type': 'tool_call'}]
========== tool ==========
1628894.626777442
========== ai ==========
Post decem annos, si depositum unius miliones cum 5% annuo interesse habes, habebis $1,628,894.63.


El modelo de LangGraph es muy flexible y permite insertar lo que uno necesite en cualquier parte del flujo, como filtrar la historia, ir resumiéndola (para no pasarse de _tokens_ o reducir posibilidades de confusión), etc. Una vez más, ejercicio para el lector.

Con esto ya cerramos el capítulo de agentes. Hay suficiente material para construir experiencias muy poderosas y productos bastante atractivos.

## Interfaces de Prototipo
Hasta ahora hemos interactuado con nuestro modelo/chatbot/agente con código. Podríamos armar una aplicación web, o conectarlo con la API de Slack/WhatsApp/Telegram/Discord/... pero queremos una forma de probarlo rápido! Una opción fácil y potente es Gradio. (Hay otras similares como Streamlit por si quieren explorar)

Ejemplo basado en [esta guía de Gradio](https://www.gradio.app/guides/creating-a-custom-chatbot-with-blocks)

In [38]:
import gradio as gr

with gr.Blocks() as app:
    # UI components
    chatbot = gr.Chatbot()
    msg = gr.Textbox()

    # Event handler, inputs: user message, and current history
    def display_user_message(message, history):
        # let's just add the user message to the "chatbot" history for now
        new_history = history + [(message, None)]
        # outputs: user message (so we can clear the input textbox) and the new message history
        return "", new_history        

    # Another event handler, inputs: current history
    def process_message(history):
        # let's actually call our last agent
        config = {"configurable":{"thread_id": "gradio"}}
        message = history[-1][0]
        result = agent.invoke({"messages":[HumanMessage(message)]}, config=config)
        agent_response = result["messages"][-1].content
        # now let's add it to the "chatbot" history
        history[-1][1] = agent_response
        return history
    
    # When submitting (enter), run `process_message`
    # with msg and chatbot as its inputs,
    # and take the outputs and put them into msg and chatbot
    msg.submit(
        fn=display_user_message,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot],
        queue=False, # We don't want to queue multiple events
    ).then( # chain the second event handler
        fn=process_message,
        inputs=chatbot,
        outputs=chatbot,
        queue=False
    )

app.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
